<a href="https://colab.research.google.com/github/elangbijak4/Riset-AI-Medis/blob/main/Training_Model_2_Deteksi_Foto_X_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Ganti dengan path folder dataset kamu
DATA_DIR = "/content/drive/MyDrive/datasets/RSNA"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Ganti dengan path folder dataset kamu
DATA_DIR = "/content/drive/MyDrive/datasets/RSNA"

In [3]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [4]:
# Split otomatis: train/val/test dari folder
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

In [5]:
def build_transfer_model(base_model):
    base_model.trainable = False  # Freeze layer awal

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Pilih salah satu
base_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_efficient = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Gunakan salah satu model
model = build_transfer_model(base_efficient)  # atau base_resnet

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Visualisasi
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()